# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import nltk
#nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import pickle
import warnings
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, precision_score
import os
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, make_scorer, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import nltk
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davideffiong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/davideffiong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/davideffiong/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/davideffiong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
print(os.getcwd())

/Users/davideffiong/Documents/Disaster-response-pipeline


In [8]:

# load data from database

engine = create_engine('sqlite:////Users/davideffiong/Documents/Disaster-Response-Pipeline/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM disaster_table", engine)
X = df['message'] #feature variable
Y = df.iloc[:,4:] #target variable
#    return X, Y






In [9]:
# dataframe head
df.head()


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
#print text data in row 15
print(X[15])

A Comitee in Delmas 19, Rue ( street ) Janvier, Impasse Charite #2. We have about 500 people in a temporary shelter and we are in dire need of Water, Food, Medications, Tents and Clothes. Please stop by and see us.


In [5]:
#instantiate lemmatizer
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
lemmatizer

<WordNetLemmatizer>

In [6]:
# a function to tokenize and lemmatize the text
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [7]:
# call the tokenize function and print the result
print(tokenize(X[15]))

['comitee', 'delmas', '19', 'rue', 'street', 'janvier', 'impasse', 'charite', '2', '500', 'people', 'temporary', 'shelter', 'dire', 'need', 'water', 'food', 'medication', 'tent', 'clothes', 'please', 'stop', 'see', 'u']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

##### Build a multi output classifier on Random Forest and Ada Boost classifier

In [83]:
#Random Forest Classifier pipeline

pipeline_rfc = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])




# Adaboost Classifier pipeline

pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [84]:
#split data into training and test data set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)


In [85]:
#fit ada classifier
pipeline_ada.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [86]:
#fit random forest classifier
pipeline_rfc.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [87]:
from sklearn.metrics import classification_report, make_scorer, f1_score
from sklearn.metrics import precision_recall_fscore_support

In [88]:
#function plot_scores to test the model and print the classification report 
def plot_scores(Y_test, Y_pred):
    i = 0
    for col in Y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(Y_test[col], Y_pred[:, i]))
        i = i + 1
    accuracy = (Y_pred == Y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))


In [89]:
# Prediction: the Random Forest Classifier  
Y_pred = pipeline_rfc.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.71      0.43      0.54      1550
           1       0.84      0.95      0.89      4951
           2       1.00      0.13      0.23        53

    accuracy                           0.82      6554
   macro avg       0.85      0.50      0.55      6554
weighted avg       0.81      0.82      0.80      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5415
           1       0.82      0.51      0.63      1139

    accuracy                           0.90      6554
   macro avg       0.86      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6417
           1       1.00      0.04      0.07       137

    accuracy                           0.98      6554
   macro avg       0.99      0.52      0.53      6554
weighted avg       0.98      0.98      0.97      6554

Feature 24: tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6516
           1       0.00      0.00      0.00        38

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Feature 25: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6488
           1       0.00      0.00      0.00        66

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99 

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

In [90]:
# Prediction: the ADA Classifier  
Y_pred = pipeline_ada.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.65      0.14      0.23      1550
           1       0.78      0.97      0.87      4951
           2       0.11      0.02      0.03        53

    accuracy                           0.77      6554
   macro avg       0.51      0.38      0.38      6554
weighted avg       0.74      0.77      0.71      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      5415
           1       0.75      0.51      0.61      1139

    accuracy                           0.88      6554
   macro avg       0.82      0.74      0.77      6554
weighted avg       0.88      0.88      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.14      0.04      0.06        26

    accuracy                           1.00      6554
   macro avg       

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.40      0.07      0.12        28

    accuracy                           1.00      6554
   macro avg       0.70      0.54      0.56      6554
weighted avg       0.99      1.00      0.99      6554

Feature 27: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6483
           1       0.30      0.08      0.13        71

    accuracy                           0.99      6554
   macro avg       0.65      0.54      0.56      6554
weighted avg       0.98      0.99      0.98      6554

Feature 28: other_infrastructure
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6265
           1       0.40      0.11      0.17       289

    accuracy                           0.95      6554
   macro avg       0.68      0.55      0.57      6554
weighted avg     

In [91]:
np.mean(Y_pred_test == Y_test)

related                   0.818126
request                   0.895331
offer                     0.996033
aid_related               0.783491
medical_help              0.917760
medical_products          0.950412
search_and_rescue         0.974825
security                  0.982759
military                  0.971315
child_alone               1.000000
water                     0.955294
food                      0.943241
shelter                   0.939274
clothing                  0.986726
money                     0.978639
missing_people            0.988099
refugees                  0.966890
death                     0.961092
other_aid                 0.868020
infrastructure_related    0.935459
transport                 0.957736
buildings                 0.949954
electricity               0.979707
tools                     0.994202
hospitals                 0.989930
shops                     0.995728
aid_centers               0.989167
other_infrastructure      0.955600
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [92]:
# Show parameters for random forest pipline
pipeline_rfc.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [94]:
# Show parameters for ada boost classifier pipline
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf

In [95]:

# Grid search parameters for Random Forest Classifier   
parameters_rfc = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv_rfc = GridSearchCV(pipeline_rfc, param_grid = parameters_rfc)
cv_rfc

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

In [104]:
# Create Grid search parameters for Ada boost classisfier
parameters_ada = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60]
}

cv_ada = GridSearchCV(pipeline_ada, param_grid = parameters_ada)
cv_ada

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60],
                         'tfidf__use_idf': (True, False)})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [98]:
# Fit the first tuned model
cv_rfc.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

In [99]:
# Fit the second tuned model
cv_ada.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7ff7439e7320>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60],
                         'tfidf__use_idf': (True, False)})

In [100]:
# Predicting using the first tuned model 
Y_pred = cv_rfc.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.68      0.45      0.54      1550
           1       0.84      0.93      0.88      4951
           2       0.88      0.13      0.23        53

    accuracy                           0.81      6554
   macro avg       0.80      0.50      0.55      6554
weighted avg       0.80      0.81      0.80      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      5415
           1       0.80      0.49      0.61      1139

    accuracy                           0.89      6554
   macro avg       0.85      0.73      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6516
           1       0.00      0.00      0.00        38

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Feature 25: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6488
           1       0.00      0.00      0.00        66

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Feature 26: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00 

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

In [101]:
# Predicting using the second tuned model 
Y_pred_rfc = cv_rfc.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.68      0.45      0.54      1550
           1       0.84      0.93      0.88      4951
           2       0.88      0.13      0.23        53

    accuracy                           0.81      6554
   macro avg       0.80      0.50      0.55      6554
weighted avg       0.80      0.81      0.80      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      5415
           1       0.80      0.49      0.61      1139

    accuracy                           0.89      6554
   macro avg       0.85      0.73      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6258
           1       0.75      0.08      0.15       296

    accuracy                           0.96      6554
   macro avg       0.85      0.54      0.56      6554
weighted avg       0.95      0.96      0.94      6554

Feature 22: buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6199
           1       0.79      0.13      0.22       355

    accuracy                           0.95      6554
   macro avg       0.87      0.56      0.60      6554
weighted avg       0.94      0.95      0.93      6554

Feature 23: electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6417
           1       1.00      0.02      0.04       137

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.52      6554
weighted avg       0.98     

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [102]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv_ada, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [8]:
def save_model(model, model_filepath):
    """Save model as pickle file"""
    pickle.dump(model, open(model_filepath, 'wb'))
